In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GlobalAveragePooling1D,Dense
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
dataset=["The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19."]

In [3]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(dataset)
word_index=tokenizer.word_index

In [5]:
sequences=tokenizer.texts_to_sequences(dataset)
X,y=[],[]

for seq in sequences:
    for  i,target_word in enumerate(seq):
        for j in range(max(0,i-2),min(i+3,len(seq))):
            if i!=j:
                X.append([target_word,seq[j]])
                y.append(1)

X=np.array(X)
y=np.array(y)

In [8]:
model=Sequential([
    Embedding(input_dim=len(word_index)+1,output_dim=10,input_length=2),
    GlobalAveragePooling1D(),
    Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])
model.fit(X,y,epochs=100,verbose=0)

In [44]:
target_word="interval"

if target_word not in word_index:
    print("Word isn't present")
else:
    target_embedding=model.layers[0].get_weights()[0][word_index[target_word]]

    similarities=[(word,np.dot(target_embedding,model.layers[0].get_weights()[0][index])/
                  (np.linalg.norm(target_embedding) * np.linalg.norm(model.layers[0].get_weights()[0][index])))
                 for word,index in word_index.items() if word!=target_word]

    related_word=sorted(similarities,key=lambda x: x[1],reverse=True)[:5]

    print(f"{target_word} ** {[word for word,_ in related_word]}")

interval ** ['than', 'transmission', 'successive', 'spread', 'viruses']
